# Airflow

Airflow is a tool that allows to schedule a set of processes commonly used in ETL pipelines and sometimes in ML automation. This section considers typical ways to use Airflow.

We typically are typically run airflow in the Docker container to make sure that we are working in a clean environment. Use following command to run the container.

Build the docker image described in the `airflow_files/dockerfile` and run it with the `standalone` command.

```bash
docker build -f packages/airflow_files/dockerfile .
docker run -d --rm --name airflow -p 8080:8080 -v ./:/knowledge airflow standalone
```

Image that is used as an example configured in such way to create default user with login `user` and password `user` that will be used as credentials for the airflow server.

## Configuration file

The global configuration of the airflow is stored in the special file: `airflow.cfg`. Different installations put this file in different places (as usual). Typical locations are `/opt/airflow/airflow.cfg` and `~/airflow/airflow.cfg`.

---

Any way use following command to find the location of the `airflow.cfg` on your disk.

In [9]:
!find / -name airflow.cfg

/opt/airflow/airflow.cfg


## Adding a dag

This section shows the minimal actions needed to create an airflow dag. It is based on the [Fundamental conceptse official tutorial](https://airflow.apache.org/docs/apache-airflow/stable/tutorial/fundamentals.html), but shows only the minimal command to add a dag and verify that it works.

---

First you need to identify the folder containing dags. This folder is specified by the `dogs_folder` parameter of the configuration file.

In [11]:
!cat /opt/airflow/airflow.cfg | grep dags_folder

dags_folder = /opt/airflow/dags


The following cell adds typical dag as it is supposed to be by the airflow develepers.

In [ ]:
%%writefile /opt/airflow/dags/tutorial.py
import textwrap
from datetime import datetime, timedelta

# The DAG object; we'll need this to instantiate a DAG
from airflow.models.dag import DAG

# Operators; we need this to operate!
from airflow.operators.bash import BashOperator
with DAG(
    "tutorial",
    default_args={
        "depends_on_past": False,
        "email": ["airflow@example.com"],
        "email_on_failure": False,
        "email_on_retry": False,
        "retries": 1,
        "retry_delay": timedelta(minutes=5),
    },
    description="A simple tutorial DAG",
    schedule=timedelta(days=1),
    start_date=datetime(2021, 1, 1),
    catchup=False,
    tags=["example"],
) as dag:

    # t1, t2 and t3 are examples of tasks created by instantiating operators
    t1 = BashOperator(
        task_id="print_date",
        bash_command="date",
    )

    t2 = BashOperator(
        task_id="sleep",
        depends_on_past=False,
        bash_command="sleep 5",
        retries=3,
    )
    t1.doc_md = textwrap.dedent(
        """\
    #### Task Documentation
    You can document your task using the attributes `doc_md` (markdown),
    `doc` (plain text), `doc_rst`, `doc_json`, `doc_yaml` which gets
    rendered in the UI's Task Instance Details page.
    ![img](https://imgs.xkcd.com/comics/fixing_problems.png)
    **Image Credit:** Randall Munroe, [XKCD](https://xkcd.com/license.html)
    """
    )

    dag.doc_md = __doc__  # providing that you have a docstring at the beginning of the DAG; OR
    dag.doc_md = """
    This is a documentation placed anywhere
    """  # otherwise, type it like this
    templated_command = textwrap.dedent(
        """
    {% for i in range(5) %}
        echo "{{ ds }}"
        echo "{{ macros.ds_add(ds, 7)}}"
    {% endfor %}
    """
    )

    t3 = BashOperator(
        task_id="templated",
        depends_on_past=False,
        bash_command=templated_command,
    )

    t1 >> [t2, t3]

Overwriting /opt/airflow/dags/tutorial.py


The following command causes airflow to add DAG to its databases.

In [3]:
!airflow db migrate

DB: sqlite:////opt/airflow/airflow.db
Performing upgrade to the metadata database sqlite:////opt/airflow/airflow.db
[2025-03-22T13:23:48.784+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-03-22T13:23:48.785+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
[2025-03-22T13:23:48.787+0000] {migration.py:207} INFO - Context impl SQLiteImpl.
[2025-03-22T13:23:48.787+0000] {migration.py:210} INFO - Will assume non-transactional DDL.
[2025-03-22T13:23:48.788+0000] {db.py:1675} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Database migrating done!


With `airflow dags list` you can show DAGs that are seen by the airflow.

In [ ]:
!airflow dags list

dag_id   | fileloc                       | owners  | is_paused
=========+===============================+=========+==========
tutorial | /opt/airflow/dags/tutorial.py | airflow | True     
                                                              


As a result there is a dag we have added below.